In [24]:
torch.cuda.empty_cache()

# OOS 

In [4]:
import data_loading
import numpy as np

In [2]:
percentile_rank_chars, _, chars, _,\
    returns, _, rfts, monthly_updates = data_loading.get_data_panel("../Data/raw_chars_returns_df_all_dates_yearly_fb.fthr",
                                                            "../Data/ff_rf.csv", computstat_data_present_filter=True,
                                                                              financial_firm_filter=False)

Index(['A2ME', 'AC', 'AT', 'ATO', 'B2M', 'BETA_d', 'BETA_m', 'C2A', 'CF2B',
       'CF2P', 'CTO', 'D2A', 'D2P', 'DPI2A', 'E2P', 'FC2Y', 'HIGH52', 'INV',
       'IdioVol', 'LEV', 'ME', 'NI', 'NOA', 'OA', 'OL', 'OP', 'PCM', 'PM',
       'PROF', 'Q', 'R12_2', 'R12_7', 'R2_1', 'R36_13', 'R60_13', 'RNA', 'ROA',
       'ROE', 'RVAR', 'S2P', 'SGA2S', 'SPREAD', 'SUV', 'TURN', 'VAR', 'return',
       'date', 'permno', 'monthly_update'],
      dtype='object')
['A2ME' 'AC' 'AT' 'ATO' 'B2M' 'BETA_d' 'BETA_m' 'C2A' 'CF2B' 'CF2P' 'CTO'
 'D2A' 'D2P' 'DPI2A' 'E2P' 'FC2Y' 'HIGH52' 'INV' 'IdioVol' 'LEV' 'ME' 'NI'
 'NOA' 'OA' 'OL' 'OP' 'PCM' 'PM' 'PROF' 'Q' 'R12_2' 'R12_7' 'R2_1'
 'R36_13' 'R60_13' 'RNA' 'ROA' 'ROE' 'RVAR' 'S2P' 'SGA2S' 'SPREAD' 'SUV'
 'TURN' 'VAR']


100%|████████████████████████████████████████████████████████████████████████████| 648/648 [03:06<00:00,  3.48it/s]


In [5]:
excess_returns = returns - np.expand_dims(np.array(rfts), axis=1)

In [6]:
excess_returns.shape

(648, 22630)

In [ ]:
rft_map = {i: rf for i, rf in enumerate(rfts)}

In [ ]:
import numpy as np

In [7]:
any_present = np.any(~np.isnan(percentile_rank_chars), axis=2)
present_counts = np.sum(np.sum(np.logical_and(~np.isnan(percentile_rank_chars),
                                             np.expand_dims(any_present, axis=2)), axis=0), axis=0)
del any_present

In [8]:
ordering = np.argsort(present_counts)[::-1]
print(chars[ordering])
percentile_rank_chars = percentile_rank_chars[:,:,ordering]
chars = chars[ordering]

['ME' 'R2_1' 'D2P' 'SPREAD' 'RVAR' 'VAR' 'IdioVol' 'TURN' 'SUV' 'R12_7'
 'CF2P' 'E2P' 'R12_2' 'S2P' 'B2M' 'CF2B' 'AT' 'A2ME' 'Q' 'PROF' 'PM' 'PCM'
 'BETA_m' 'C2A' 'OL' 'ROE' 'CTO' 'ATO' 'LEV' 'NOA' 'ROA' 'RNA' 'OP' 'NI'
 'INV' 'BETA_d' 'R36_13' 'D2A' 'FC2Y' 'SGA2S' 'OA' 'AC' 'HIGH52' 'R60_13'
 'DPI2A']


In [9]:
del percentile_rank_chars

In [10]:
def load_imputation(name, full=False):
    base_path = '../Data/'
    result_file_name = base_path + name + '.npz'
    res = np.load(result_file_name)
    if not full:
        return res['data']
    else:
        return res['data'], res['dates'], res['permnos'], res['chars']

In [11]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
del masked_data, eval_data, gamma_ts

In [13]:
import importlib
import eval_fn
importlib.reload(data_loading)
importlib.reload(eval_fn)

<module 'eval_fn' from '/home/selwin_p_george/CS 236 Project/Code/eval_fn.py'>

In [37]:
return_lag = 3
gamma_ts = load_imputation('MAR_xs_factors_cs236_2')
print("loaded")
first_masked_data = load_imputation('MAR_fit_data_cs236')[:, :, ordering]
train_data_seq = data_loading.get_data(first_masked_data[:400], min_chars_present=10, fill_val=-1, 
                                   gamma_ts=gamma_ts[:400], ordered=True,
                                        return_panel=excess_returns[return_lag:400+return_lag],
                                      t_start=return_lag)

del first_masked_data
print("loaded")
masked_data = load_imputation('MAR_fit_data_cs236_2')[:, :, ordering]
print("loaded")
eval_data = load_imputation('MAR_eval_data_cs236_2')[:, :, ordering]
print("loaded")
eval_data_seq = data_loading.get_data(masked_data[400:500], min_chars_present=10, fill_val=-1, 
                                   gamma_ts=gamma_ts[400:500], ordered=False,
                    oos_mask=~np.isnan(eval_data[400:500]), oos_eval_data=eval_data[400:500],
                                        return_panel=excess_returns[400+return_lag:500+return_lag],
                                      t_start=return_lag+400)
test_data_seq = data_loading.get_data(masked_data[500:-return_lag], min_chars_present=10, fill_val=-1, 
                                   gamma_ts=gamma_ts[500:-return_lag], ordered=False,
                    oos_mask=~np.isnan(eval_data[500:-return_lag]), oos_eval_data=eval_data[500:-return_lag],
                                        return_panel=excess_returns[500+return_lag:],
                                      t_start=return_lag+500)
del masked_data
del eval_data

loaded
1578149


100%|████████████████████████████████████████████████████████████████████████████| 400/400 [00:10<00:00, 39.71it/s]


loaded
loaded
loaded


100%|████████████████████████████████████████████████████████████████████████████| 145/145 [00:02<00:00, 66.23it/s]


In [15]:
len(train_data_seq), len(eval_data_seq), len(test_data_seq)

(8674, 2924, 2669)

In [16]:
importlib.reload(data_loading)

<module 'data_loading' from '/home/selwin_p_george/CS 236 Project/Code/data_loading.py'>

In [39]:
chunk_size = 20
batch_size = 100

all_dataset = data_loading.ListDatasetWithFactors(train_data_seq, batch_size=chunk_size,return_returns=True)
train_data_loader = data_loading.DataLoader(all_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

all_dataset = data_loading.ListDatasetWithFactors(eval_data_seq, batch_size=chunk_size,
                                                 return_oos_data=True,return_returns=True)
eval_data_loader = data_loading.DataLoader(all_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

all_dataset = data_loading.ListDatasetWithFactors(test_data_seq, batch_size=chunk_size,
                                                 return_oos_data=True,return_returns=True)
test_data_loader = data_loading.DataLoader(all_dataset, batch_size=batch_size, shuffle=True, num_workers=4)


In [18]:
import models
import loss_functions
from tqdm.notebook import tqdm

In [19]:
import numpy as np
import csv
import pandas as pd
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
import time

In [20]:
import models
import loss_functions
importlib.reload(models)

<module 'models' from '/home/selwin_p_george/CS 236 Project/Code/models.py'>

In [21]:
import importlib
importlib.reload(models)
importlib.reload(loss_functions)

<module 'loss_functions' from '/home/selwin_p_george/CS 236 Project/Code/loss_functions.py'>

# Returns

In [148]:
del eval_data_loader, ar_model, optimizer

In [46]:
torch.cuda.memory_reserved(0)

2097152

In [43]:
gc.collect()
torch.cuda.empty_cache()

In [30]:
import gc

In [125]:
def markowitz(factors):
    cov = np.cov(factors.T)
    mu = np.mean(factors, axis=0)
    print(cov.shape, mu.shape)
    return np.linalg.solve(cov, mu) 

In [109]:
for i in is_hidden:
    np.array(eval_returns[i])
    np.array(eval_hidden[i])

for i in eval_returns:
    np.array(eval_returns[i])
    np.array(eval_hidden[i])

In [91]:
importlib.reload(models)

<module 'models' from '/home/selwin_p_george/CS 236 Project/Code/models.py'>

In [207]:
ar_model = models.Beta_NADE(hidden_state_dim=0, latent_dimensions=5, predict_returns=True)
exp_batch_size=50

num_epochs = 41
eval_freq = 10

optimizer = torch.optim.AdamW(ar_model.parameters(), lr=0.01, weight_decay=0.1)


scheduler = MultiStepLR(optimizer, [20, 35], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')

ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):

        ts_idxs, stock_idxs, data, mask, ordered_mask, factors, return_sample = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)

        C_train = data.transpose_(0, 1).float().to(device)[:-1]

        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        return_sample = return_sample.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]

        factors = factors.transpose_(0, 1).float().to(device)[1:]

        train_input = torch.cat([C_train, C_mask, factors], axis=2)


        if train_input.shape[1] > 0:

            batch_size, batch_length, dim = C_train.shape
            model_input = C_train.reshape(batch_size * batch_length, dim)

            C_ordered_mask_reshape = C_ordered_mask.reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train.reshape(batch_size * batch_length, 45)
            return_sample_reshape = return_sample.reshape(batch_size * batch_length, 1)

            alphas_pred, betas_pred, return_mus, return_sigmas = ar_model(model_input)

            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1, ret_mus=return_mus, ret_sigmas=return_sigmas,
                             return_gts=return_sample_reshape).mean()

            net_loss = ar_loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())

#             if i % 100 == 0:
#                 print(loss)
#                 print(ar_loss)

#             if i % 10 == 0:
    #             torch.nn.utils.clip_grad_norm_(model.parameters(), 50)
            optimizer.step()
            ar_model.zero_grad()
            optimizer.zero_grad()   

    print(f'epoch {epoch}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch + 1) % eval_freq == 0 or np.mean(ar_losses) <= -15:
        print(eval_model_is(exp_batch_size, None, ar_model, eval_data_loader, return_metrics=True, expect_oos_data=True))
        ar_model.predict_returns = False
        print(eval_model_oos(exp_batch_size, None, ar_model, eval_data_loader))
        ar_model.predict_returns = True

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True) torch.Size([46, 45])
torch device is cuda:0
epoch 0


  0%|          | 0/731 [00:00<?, ?it/s]

epoch 0, avg ar model loss 45.24089431762695
epoch 0 took 0.4922452251116435 minutes
epoch 1


  0%|          | 0/731 [00:00<?, ?it/s]

epoch 1, avg ar model loss -0.30153363943099976
epoch 1 took 0.4990278482437134 minutes
epoch 2


  0%|          | 0/731 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [206]:
is_hidden, is_returns = get_factors_returns(model=None, ar_model=ar_model, 
                                            data_loader=train_data_loader, expect_oos_data=False)

eval_hidden, eval_returns = get_factors_returns(model=None, ar_model=ar_model, 
                                            data_loader=eval_data_loader, expect_oos_data=True)


is_factors = get_factors(is_hidden, is_returns).squeeze()

eval_factors = get_factors(eval_hidden, eval_returns).squeeze()

weights = markowitz(is_factors.squeeze())

np.mean(is_factors @ weights) / np.std(is_factors @ weights),\
    np.mean(eval_factors @ weights) / np.std(eval_factors @ weights)

  0%|          | 0/731 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

(5, 5) (5,)


(0.3402409271881175, 0.10340032069601024)

In [226]:
importlib.reload(eval_fn)
importlib.reload(models)


<module 'models' from '/home/selwin_p_george/CS 236 Project/Code/models.py'>

In [213]:
import eval_fn

In [53]:
torch.manual_seed(0)
exp_batch_size = 100
# model = models.XSCharLSTM(input_dim=96, hidden_dims=[30, 40], num_layers=4, batch_size=exp_batch_size)

# ar_model = models.Beta_NADE(hidden_state_dim=30, latent_dimensions=30, predict_returns=True)

num_epochs = 10

eval_freq = 1

# optimizer = torch.optim.AdamW(list(model.parameters()) + list(ar_model.parameters()), lr=0.01,
#                              weight_decay=0.01)
optimizer = torch.optim.AdamW(ar_model.parameters(), lr=0.01,
                             weight_decay=0.01)


scheduler = MultiStepLR(optimizer, [30], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        ts_idxs, stock_idxs, data, mask, ordered_mask, factors, return_sample = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        return_sample = return_sample.float().to(device).transpose_(0, 1)[:-1]
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == exp_batch_size:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
#             loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
#             print(hidden_out.shape, C_train.shape)
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_dim, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_dim * batch_length, dim)
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_dim * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_dim * batch_length, 45)
            return_sample_reshape = return_sample[1:].reshape(batch_dim * batch_length, 1)
            
            alphas_pred, betas_pred, return_mus, return_sigmas = ar_model(model_input)
            
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1, ret_mus=return_mus, ret_sigmas=return_sigmas,
                             return_gts=return_sample_reshape).mean()
            
#             net_loss = ar_loss + loss
            net_loss = ar_loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            

            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch + 1) % eval_freq == 0:
#         print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, train_data_loader, return_metrics=True, expect_oos_data=False))
#         print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, eval_data_loader, return_metrics=True, expect_oos_data=True))
        print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, test_data_loader, return_metrics=True, expect_oos_data=True))
#         ar_model.predict_returns = False
#         print(eval_fn.eval_model_oos(chars, exp_batch_size ,model, ar_model, eval_data_loader))
#         ar_model.predict_returns = True
#         print(eval_fn.eval_ts_model_oos(chars, exp_batch_size,model, eval_data_loader))

torch device is cuda:0
epoch 0


  0%|          | 0/370 [00:00<?, ?it/s]

epoch 0 avg lstm loss -25.44904327392578, avg ar model loss -75.0473861694336
epoch 0 took 0.42998618284861245 minutes


  0%|          | 0/77 [00:00<?, ?it/s]

([('ME', 0.028430201), ('R2_1', 0.0068970406), ('D2P', 0.006089899), ('SPREAD', 0.007317093), ('RVAR', 0.02488415), ('VAR', 0.022310214), ('IdioVol', 0.009900408), ('TURN', 0.009013364), ('SUV', 0.0075842696), ('R12_7', 0.011956483), ('CF2P', 0.009550339), ('E2P', 0.012644095), ('R12_2', 0.01072837), ('S2P', 0.017609159), ('B2M', 0.009717444), ('CF2B', 0.0076990467), ('AT', 0.02976493), ('A2ME', 0.036694176), ('Q', 0.018211769), ('PROF', 0.012442904), ('PM', 0.009598165), ('PCM', 0.010238426), ('BETA_m', 0.0121489335), ('C2A', 0.007945801), ('OL', 0.017189058), ('ROE', 0.010375021), ('CTO', 0.016877003), ('ATO', 0.011355337), ('LEV', 0.009615761), ('NOA', 0.008768651), ('ROA', 0.012248337), ('RNA', 0.006668899), ('OP', 0.011724933), ('NI', 0.008366426), ('INV', 0.008878861), ('BETA_d', 0.012716819), ('R36_13', 0.014101563), ('D2A', 0.012403686), ('FC2Y', 0.017296191), ('SGA2S', 0.011085946), ('OA', 0.0071569923), ('AC', 0.01192571), ('HIGH52', 0.009070999), ('R60_13', 0.014208964), ('D

  0%|          | 0/370 [00:00<?, ?it/s]

epoch 1 avg lstm loss -25.44755744934082, avg ar model loss -76.22473907470703
epoch 1 took 0.4152739723523458 minutes


  0%|          | 0/77 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
@ 4 [0.001977953470518154, 0.07228942832286273, 0.060806419080847954, -0.011563901668093778, 0.00026002147163928355, 0.06765969975657786, 0.14694436342760686, 0.11438107251132079, 0.07199162741893389, 0.22428494061929422, 0.12180594761727552]

In [290]:
importlib.reload(eval_fn)

<module 'eval_fn' from '/home/selwin_p_george/CS 236 Project/Code/eval_fn.py'>

In [54]:
print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, train_data_loader,
                            return_metrics=True, expect_oos_data=False))
print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, eval_data_loader,
                            return_metrics=True, expect_oos_data=True))
print(eval_fn.eval_model_is(chars, exp_batch_size, model, ar_model, test_data_loader,
                            return_metrics=True, expect_oos_data=True))

  0%|          | 0/370 [00:00<?, ?it/s]

([('ME', 0.008855535), ('R2_1', 0.005697643), ('D2P', 0.006664096), ('SPREAD', 0.006592712), ('RVAR', 0.010827155), ('VAR', 0.010680204), ('IdioVol', 0.0073566106), ('TURN', 0.008304125), ('SUV', 0.006479653), ('R12_7', 0.00808746), ('CF2P', 0.008772258), ('E2P', 0.0065189614), ('R12_2', 0.006516874), ('S2P', 0.010319276), ('B2M', 0.008913608), ('CF2B', 0.0059681376), ('AT', 0.009221396), ('A2ME', 0.011313069), ('Q', 0.008509975), ('PROF', 0.0073401593), ('PM', 0.006816445), ('PCM', 0.0061151786), ('BETA_m', 0.007682451), ('C2A', 0.007110572), ('OL', 0.009288064), ('ROE', 0.008334165), ('CTO', 0.00988218), ('ATO', 0.0074910237), ('LEV', 0.00793094), ('NOA', 0.0064094993), ('ROA', 0.007166346), ('RNA', 0.0052977544), ('OP', 0.008012586), ('NI', 0.0067808195), ('INV', 0.00845476), ('BETA_d', 0.011215329), ('R36_13', 0.00838214), ('D2A', 0.00789875), ('FC2Y', 0.00909588), ('SGA2S', 0.0068780496), ('OA', 0.0069415015), ('AC', 0.007709105), ('HIGH52', 0.0072687874), ('R60_13', 0.011510137),

  0%|          | 0/64 [00:00<?, ?it/s]

([('ME', 0.023574956), ('R2_1', 0.0060577486), ('D2P', 0.0067644934), ('SPREAD', 0.0070135877), ('RVAR', 0.014069514), ('VAR', 0.01309053), ('IdioVol', 0.00834095), ('TURN', 0.00811267), ('SUV', 0.006484365), ('R12_7', 0.009563644), ('CF2P', 0.010170094), ('E2P', 0.009615251), ('R12_2', 0.008498814), ('S2P', 0.014604301), ('B2M', 0.009639329), ('CF2B', 0.006657991), ('AT', 0.022795033), ('A2ME', 0.02177308), ('Q', 0.012942621), ('PROF', 0.00855805), ('PM', 0.0075769103), ('PCM', 0.0076797763), ('BETA_m', 0.008974282), ('C2A', 0.007126991), ('OL', 0.010616143), ('ROE', 0.009237268), ('CTO', 0.011900856), ('ATO', 0.0083425455), ('LEV', 0.009050978), ('NOA', 0.0069432645), ('ROA', 0.009631499), ('RNA', 0.005374136), ('OP', 0.009074297), ('NI', 0.0071789715), ('INV', 0.009012678), ('BETA_d', 0.012404959), ('R36_13', 0.010076339), ('D2A', 0.009472845), ('FC2Y', 0.013879378), ('SGA2S', 0.0076361243), ('OA', 0.0071610636), ('AC', 0.008647786), ('HIGH52', 0.007755195), ('R60_13', 0.013583305),

  0%|          | 0/77 [00:00<?, ?it/s]

([('ME', 0.023513164), ('R2_1', 0.006032398), ('D2P', 0.0065296274), ('SPREAD', 0.007177128), ('RVAR', 0.01952104), ('VAR', 0.018112466), ('IdioVol', 0.008639014), ('TURN', 0.0082338685), ('SUV', 0.006549479), ('R12_7', 0.009524478), ('CF2P', 0.010289551), ('E2P', 0.009681828), ('R12_2', 0.008537712), ('S2P', 0.014661506), ('B2M', 0.009631356), ('CF2B', 0.006654524), ('AT', 0.023182243), ('A2ME', 0.026068285), ('Q', 0.014435117), ('PROF', 0.009324116), ('PM', 0.008073493), ('PCM', 0.008778263), ('BETA_m', 0.009323122), ('C2A', 0.0069631324), ('OL', 0.011486181), ('ROE', 0.009373486), ('CTO', 0.012633923), ('ATO', 0.008830623), ('LEV', 0.009046309), ('NOA', 0.0071038315), ('ROA', 0.010023458), ('RNA', 0.0052379104), ('OP', 0.009554217), ('NI', 0.0076110284), ('INV', 0.009000656), ('BETA_d', 0.01208373), ('R36_13', 0.0101788845), ('D2A', 0.010709635), ('FC2Y', 0.014697587), ('SGA2S', 0.009498118), ('OA', 0.0072221565), ('AC', 0.009426906), ('HIGH52', 0.007480727), ('R60_13', 0.014865014)

In [241]:
is_hidden, is_returns = eval_fn.get_factors_returns(exp_batch_size, model=model, ar_model=ar_model, 
                                            data_loader=train_data_loader, expect_oos_data=False)

eval_hidden, eval_returns = eval_fn.get_factors_returns(exp_batch_size, model=model, ar_model=ar_model, 
                                            data_loader=eval_data_loader, expect_oos_data=True)


is_factors = eval_fn.get_factors(is_hidden, is_returns).squeeze()

eval_factors = eval_fn.get_factors(eval_hidden, eval_returns).squeeze()

weights = markowitz(is_factors.squeeze())

np.mean(is_factors @ weights) / np.std(is_factors @ weights),\
    np.mean(eval_factors @ weights) / np.std(eval_factors @ weights)

  0%|          | 0/366 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

(5, 5) (5,)


(0.138095236684946, 0.10802225494038135)

# OOS validation

49

In [248]:
importlib.reload(eval_fn)

<module 'eval_fn' from '/home/selwin_p_george/CS 236 Project/Code/eval_fn.py'>

In [29]:
exp_batch_size = 100
model = models.XSCharLSTM(input_dim=96, hidden_dims=[30, 30], num_layers=4, batch_size=exp_batch_size)

num_epochs = 41
eval_freq = 5

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


scheduler = MultiStepLR(optimizer, [30], gamma=0.1, last_epoch=-1)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        ts_idxs, stock_idxs, data, mask, ordered_mask, factors, return_sample = data
        assert not torch.isnan(data).any()
#         assert not (data <= 0).any(), torch.min(data)
#         print(data.shape)
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        if train_input.shape[1] == exp_batch_size and train_input.shape[2] > 0:
            alpha_pred, beta_pred, hidden_out = model(train_input)
#             print(hidden_out.shape)

            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()

            loss.backward()
            losses.append(loss.data.cpu().detach().numpy())

            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       


    print(f'epoch {epoch} avg loss {np.mean(losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch) % eval_freq == 0:
#         print("train is", eval_fn.eval_ts_model_is(chars, exp_batch_size, model, train_data_loader))
#         print("eval is", eval_fn.eval_ts_model_is(chars, exp_batch_size, model, eval_data_loader))
        print("eval oos", eval_fn.eval_ts_model_oos(chars, exp_batch_size, model, eval_data_loader))
        print("test oos", eval_fn.eval_ts_model_oos(chars, exp_batch_size, model, test_data_loader))
        

torch device is cuda:0
epoch 0


  0%|          | 0/367 [00:00<?, ?it/s]

epoch 0 avg loss 15.583824157714844
epoch 0 took 0.1231488307317098 minutes


  0%|          | 0/367 [00:00<?, ?it/s]

train is ([('ME', 0.20462418), ('R2_1', 0.2837318), ('D2P', 0.24477395), ('SPREAD', 0.18411347), ('RVAR', 0.14534199), ('VAR', 0.14846684), ('IdioVol', 0.15650497), ('TURN', 0.27052894), ('SUV', 0.28476137), ('R12_7', 0.26853973), ('CF2P', 0.24004841), ('E2P', 0.2607011), ('R12_2', 0.25920385), ('S2P', 0.21611597), ('B2M', 0.19027111), ('CF2B', 0.28137174), ('AT', 0.18840744), ('A2ME', 0.081389695), ('Q', 0.085096054), ('PROF', 0.2870594), ('PM', 0.24301358), ('PCM', 0.27915263), ('BETA_m', 0.2702548), ('C2A', 0.26354107), ('OL', 0.28260407), ('ROE', 0.25338334), ('CTO', 0.28518856), ('ATO', 0.28645617), ('LEV', 0.24650545), ('NOA', 0.28789398), ('ROA', 0.23578961), ('RNA', 0.24221665), ('OP', 0.2764733), ('NI', 0.28117236), ('INV', 0.27354708), ('BETA_d', 0.26724747), ('R36_13', 0.2608023), ('D2A', 0.2846741), ('FC2Y', 0.2596851), ('SGA2S', 0.25768143), ('OA', 0.28704214), ('AC', 0.29069173), ('HIGH52', 0.24122564), ('R60_13', 0.25474936), ('DPI2A', 0.28291437)], 0.24388792)


...

epoch 21 avg loss -22.26443099975586
epoch 21 took 0.12512712478637694 minutes
epoch 22


  0%|          | 0/367 [00:00<?, ?it/s]

epoch 22 avg loss -23.910919189453125
epoch 22 took 0.12596531709035239 minutes
epoch 23


  0%|          | 0/367 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [34]:
exp_batch_size = 100
ar_model = models.Beta_NADE(hidden_state_dim=0, latent_dimensions=30)

num_epochs = 41
eval_freq = 1

optimizer = torch.optim.AdamW(ar_model.parameters(), lr=0.01,
                             weight_decay=0.01)

scheduler = MultiStepLR(optimizer, [20, 35], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        ts_idxs, stock_idxs, data, mask, ordered_mask, factors, return_sample = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] > 0:
            
            batch_size, batch_length, dim = C_train.shape
            model_input = C_train.reshape(batch_size * batch_length, dim)
            
            C_ordered_mask_reshape = C_ordered_mask.reshape(batch_size * batch_length, 45)
            C_train_reshape = C_train.reshape(batch_size * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            
            net_loss = ar_loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            

            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

    print(f'epoch {epoch}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch + 1) % eval_freq == 0:

#         print("train is", eval_fn.eval_model_is(chars, exp_batch_size, None, ar_model, train_data_loader))
        print("eval oos", eval_fn.eval_model_oos(chars, exp_batch_size, None, ar_model, eval_data_loader))
        print("test oos", eval_fn.eval_model_oos(chars, exp_batch_size, None, ar_model, test_data_loader))

Parameter containing:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]], requires_grad=True) torch.Size([45, 45])
torch device is cuda:0
epoch 0


  0%|          | 0/367 [00:00<?, ?it/s]

epoch 0, avg ar model loss 21.132387161254883
epoch 0 took 0.3734210809071859 minutes


  0%|          | 0/63 [00:00<?, ?it/s]

eval oos ([('ME', 0.26400554), ('R2_1', 0.28582013), ('D2P', 0.22921543), ('SPREAD', 0.26570666), ('RVAR', 0.11779517), ('VAR', 0.108356304), ('IdioVol', 0.12682123), ('TURN', 0.20516038), ('SUV', 0.27478975), ('R12_7', 0.27662283), ('CF2P', 0.14684032), ('E2P', 0.21235281), ('R12_2', 0.115088515), ('S2P', 0.2725678), ('B2M', 0.20063977), ('CF2B', 0.2579328), ('AT', 0.15077086), ('A2ME', 0.09117512), ('Q', 0.16427444), ('PROF', 0.24424551), ('PM', 0.15542577), ('PCM', 0.23026715), ('BETA_m', 0.25838113), ('C2A', 0.28103545), ('OL', 0.0936213), ('ROE', 0.22381441), ('CTO', 0.09475306), ('ATO', 0.18122514), ('LEV', 0.25888768), ('NOA', 0.27354398), ('ROA', 0.19402462), ('RNA', 0.12596945), ('OP', 0.21742477), ('NI', 0.28308764), ('INV', 0.18242493), ('BETA_d', 0.14880358), ('R36_13', 0.24941748), ('D2A', 0.21591991), ('FC2Y', 0.12505284), ('SGA2S', 0.10998739), ('OA', 0.27956313), ('AC', 0.18924841), ('HIGH52', 0.23083307), ('R60_13', 0.18059899), ('DPI2A', 0.28129378)], 0.21115443)


...

test oos ([('ME', 0.11421501), ('R2_1', 0.1780236), ('D2P', 0.18368852), ('SPREAD', 0.20709813), ('RVAR', 0.04897987), ('VAR', 0.05328819), ('IdioVol', 0.17435396), ('TURN', 0.21154724), ('SUV', 0.21135233), ('R12_7', 0.17135474), ('CF2P', 0.16590971), ('E2P', 0.15047115), ('R12_2', 0.16147575), ('S2P', 0.07714967), ('B2M', 0.19429271), ('CF2B', 0.19554722), ('AT', 0.071340516), ('A2ME', 0.07589357), ('Q', 0.11366249), ('PROF', 0.20700666), ('PM', 0.15428446), ('PCM', 0.1840623), ('BETA_m', 0.18125112), ('C2A', 0.15836076), ('OL', 0.1221919), ('ROE', 0.12788023), ('CTO', 0.0609608), ('ATO', 0.19996355), ('LEV', 0.21079655), ('NOA', 0.15153426), ('ROA', 0.1685353), ('RNA', 0.15993331), ('OP', 0.18266313), ('NI', 0.19861813), ('INV', 0.2001739), ('BETA_d', 0.20602554), ('R36_13', 0.18740107), ('D2A', 0.18973504), ('FC2Y', 0.18587019), ('SGA2S', 0.17051172), ('OA', 0.18542957), ('AC', 0.17952399), ('HIGH52', 0.18397963), ('R60_13', 0.16008836), ('DPI2A', 0.19711225)], 0.16783509)
epoch 3


  0%|          | 0/367 [00:00<?, ?it/s]

epoch 3, avg ar model loss -69.39411926269531
epoch 3 took 0.37095653216044105 minutes


  0%|          | 0/63 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [256]:
print("train is", eval_fn.eval_model_is(chars, exp_batch_size, None, ar_model, train_data_loader))

  0%|          | 0/366 [00:00<?, ?it/s]

train is ([('A2ME', 0.013976982), ('AC', 0.005806589), ('AT', 0.010489577), ('ATO', 0.0053844713), ('B2M', 0.0056127436), ('BETA_d', 0.0053472123), ('BETA_m', 0.011935077), ('C2A', 0.010163785), ('CF2B', 0.015316576), ('CF2P', 0.0068335677), ('CTO', 0.014923305), ('D2A', 0.011769143), ('D2P', 0.0058370223), ('DPI2A', 0.0050148973), ('E2P', 0.01504475), ('FC2Y', 0.01381663), ('HIGH52', 0.013624067), ('INV', 0.020456163), ('IdioVol', 0.0084419735), ('LEV', 0.007519309), ('ME', 0.00839054), ('NI', 0.009119356), ('NOA', 0.22738501), ('OA', 0.013912534), ('OL', 0.012288307), ('OP', 0.015225656), ('PCM', 0.012945927), ('PM', 0.014856877), ('PROF', 0.009664656), ('Q', 0.005674054), ('R12_2', 0.012534675), ('R12_7', 0.009883886), ('R2_1', 0.011054655), ('R36_13', 0.25573757), ('R60_13', 0.021404624), ('RNA', 0.013092114), ('ROA', 0.23576026), ('ROE', 0.006940409), ('RVAR', 0.012545756), ('S2P', 0.014058761), ('SGA2S', 0.009228436), ('SPREAD', 0.008892003), ('SUV', 0.0062662964), ('TURN', 0.009

In [253]:
importlib.reload(eval_fn)

<module 'eval_fn' from '/home/selwin_p_george/CS 236 Project/Code/eval_fn.py'>

In [35]:
exp_batch_size = 100
# model = models.XSCharLSTM(input_dim=96, hidden_dims=[30, 30], num_layers=4, batch_size=exp_batch_size)

ar_model = models.Beta_NADE(hidden_state_dim=30, latent_dimensions=30)

num_epochs = 5
eval_freq = 1

# optimizer = torch.optim.AdamW(list(model.parameters()) + list(ar_model.parameters()), lr=0.01,
#                              weight_decay=0.01)
optimizer = torch.optim.AdamW(ar_model.parameters(), lr=0.01,
                             weight_decay=0.1)


scheduler = MultiStepLR(optimizer, [30], gamma=0.1, last_epoch=-1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'torch device is {device}')
model = model.to(device)
ar_model = ar_model.to(device)
loss_fn = loss_functions.beta_ll_loss


#basic training loop for local development
for epoch in range(num_epochs):
    epoch_start = time.time()
    losses = []
    ar_losses = []
    # Forward pass
    print(f'epoch {epoch}')
    for i, data in enumerate(tqdm(train_data_loader)):
        ts_idxs, stock_idxs, data, mask, ordered_mask, factors, return_sample = data
        assert not torch.isnan(data).any()
#         print(data, mask)
        assert not torch.logical_and(data <= 0, ~mask).any(), torch.min(data * mask)
        
        C_train = data.transpose_(0, 1).float().to(device)[:-1]
        
        C_mask = mask.float().to(device).transpose_(0, 1)[:-1]
        C_ordered_mask = ordered_mask.float().to(device).transpose_(0, 1)[:-1]
        
        factors = factors.transpose_(0, 1).float().to(device)[1:]
        
        train_input = torch.cat([C_train, C_mask, factors], axis=2)
        
      
        if train_input.shape[1] == exp_batch_size:
            alpha_pred, beta_pred, hidden_out = model(train_input)
            loss = loss_fn(C_train[1:], alpha_pred[:-1], beta_pred[:-1], mask=C_mask[1:]).mean()
            assert not torch.isnan(loss).any()
            
#             loss.backward()
            losses.append(loss.data.detach().cpu().numpy())
            
#             print(hidden_out.shape, C_train.shape)
            model_input = torch.cat([hidden_out[:-1].detach(), C_train[1:]], axis=2)
            batch_dim, batch_length, dim = model_input.shape
            model_input = model_input.reshape(batch_dim * batch_length, dim)
            
            C_ordered_mask_reshape = C_ordered_mask[1:].reshape(batch_dim * batch_length, 45)
            C_train_reshape = C_train[1:].reshape(batch_dim * batch_length, 45)

            alphas_pred, betas_pred = ar_model(model_input)
            
            ar_loss = loss_fn(C_train_reshape,
                              alphas_pred, betas_pred, mask=C_ordered_mask_reshape,
                             reduce_axis=1).mean()
            
#             net_loss = ar_loss + loss
            net_loss = ar_loss
            net_loss.backward()
            ar_losses.append(ar_loss.data.detach().cpu().numpy())
            

            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()       

    print(f'epoch {epoch} avg lstm loss {np.mean(losses)}, avg ar model loss {np.mean(ar_losses)}')
    print(f'epoch {epoch} took {(time.time() - epoch_start)/60.0} minutes')
    scheduler.step()
    if (epoch + 1) % eval_freq == 0:
        print("eval oos", eval_fn.eval_model_oos(chars, exp_batch_size, model, ar_model, eval_data_loader))
        print("test oos", eval_fn.eval_model_oos(chars, exp_batch_size, model, ar_model, test_data_loader))

Parameter containing:
tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.]], requires_grad=True) torch.Size([45, 75])
torch device is cuda:0
epoch 0


  0%|          | 0/367 [00:00<?, ?it/s]

epoch 0 avg lstm loss -25.458040237426758, avg ar model loss 20.773578643798828
epoch 0 took 0.3939926862716675 minutes


  0%|          | 0/63 [00:00<?, ?it/s]

eval oos ([('ME', 0.11557899), ('R2_1', 0.2828822), ('D2P', 0.25621563), ('SPREAD', 0.20610106), ('RVAR', 0.083032496), ('VAR', 0.07997706), ('IdioVol', 0.09153421), ('TURN', 0.23710197), ('SUV', 0.2818259), ('R12_7', 0.1840996), ('CF2P', 0.15172286), ('E2P', 0.11375432), ('R12_2', 0.12520052), ('S2P', 0.12048655), ('B2M', 0.18432994), ('CF2B', 0.25605598), ('AT', 0.10248418), ('A2ME', 0.113484204), ('Q', 0.101084754), ('PROF', 0.12630165), ('PM', 0.1287645), ('PCM', 0.14800009), ('BETA_m', 0.2319436), ('C2A', 0.12984702), ('OL', 0.12908821), ('ROE', 0.1724781), ('CTO', 0.1024023), ('ATO', 0.13093932), ('LEV', 0.15297104), ('NOA', 0.11328644), ('ROA', 0.15200529), ('RNA', 0.11486773), ('OP', 0.16860591), ('NI', 0.24893853), ('INV', 0.19132118), ('BETA_d', 0.15222718), ('R36_13', 0.1786704), ('D2A', 0.20277497), ('FC2Y', 0.11724991), ('SGA2S', 0.0867942), ('OA', 0.19587594), ('AC', 0.2904638), ('HIGH52', 0.1462593), ('R60_13', 0.18672259), ('DPI2A', 0.1977596)], 0.17167643)


...

epoch 2 avg lstm loss -25.464027404785156, avg ar model loss -64.64750671386719
epoch 2 took 0.385468320051829 minutes


  0%|          | 0/63 [00:00<?, ?it/s]

eval oos ([('ME', 0.09236818), ('R2_1', 0.20117113), ('D2P', 0.20351522), ('SPREAD', 0.18946223), ('RVAR', 0.045304563), ('VAR', 0.045304075), ('IdioVol', 0.17888866), ('TURN', 0.2006099), ('SUV', 0.1952014), ('R12_7', 0.11313843), ('CF2P', 0.1587347), ('E2P', 0.18032622), ('R12_2', 0.10317977), ('S2P', 0.056593336), ('B2M', 0.2109542), ('CF2B', 0.19724204), ('AT', 0.066172265), ('A2ME', 0.07466907), ('Q', 0.05054835), ('PROF', 0.1902591), ('PM', 0.10480217), ('PCM', 0.18462639), ('BETA_m', 0.21282817), ('C2A', 0.2133907), ('OL', 0.070440345), ('ROE', 0.16445959), ('CTO', 0.07277224), ('ATO', 0.18182927), ('LEV', 0.16230763), ('NOA', 0.1257639), ('ROA', 0.17312516), ('RNA', 0.14832012), ('OP', 0.17607222), ('NI', 0.21490331), ('INV', 0.20343553), ('BETA_d', 0.19699988), ('R36_13', 0.18426713), ('D2A', 0.17504363), ('FC2Y', 0.07995747), ('SGA2S', 0.07105912), ('OA', 0.20412399), ('AC', 0.19346479), ('HIGH52', 0.17112294), ('R60_13', 0.13658063), ('DPI2A', 0.20282091)], 0.16050151)


  0%|          | 0/76 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
print("train is", eval_fn.eval_model_is(chars, exp_batch_size, None, ar_model, train_data_loader))

In [265]:
print("eval oos", eval_fn.eval_model_oos(chars, exp_batch_size, model, ar_model, eval_data_loader))
print("test oos", eval_fn.eval_model_oos(chars, exp_batch_size, model, ar_model, test_data_loader))

  0%|          | 0/63 [00:00<?, ?it/s]

eval oos ([('A2ME', 0.10536056), ('AC', 0.17449063), ('AT', 0.21230346), ('ATO', 0.19269145), ('B2M', 0.04092196), ('BETA_d', 0.04652951), ('BETA_m', 0.22659256), ('C2A', 0.209662), ('CF2B', 0.2146566), ('CF2P', 0.21060885), ('CTO', 0.2066392), ('D2A', 0.19175221), ('D2P', 0.1961372), ('DPI2A', 0.22832267), ('E2P', 0.20253451), ('FC2Y', 0.19534673), ('HIGH52', 0.08216147), ('INV', 0.03930987), ('IdioVol', 0.047781587), ('LEV', 0.21651003), ('ME', 0.1780023), ('NI', 0.20914699), ('NOA', 0.19389413), ('OA', 0.22030994), ('OL', 0.14608136), ('OP', 0.19803634), ('PCM', 0.2168286), ('PM', 0.22026871), ('PROF', 0.20430529), ('Q', 0.21752992), ('R12_2', 0.20087689), ('R12_7', 0.20444821), ('R2_1', 0.19684927), ('R36_13', 0.22137447), ('R60_13', 0.2044911), ('RNA', 0.22181948), ('ROA', 0.16629039), ('ROE', 0.2170001), ('RVAR', 0.1911478), ('S2P', 0.1954653), ('SGA2S', 0.21906675), ('SPREAD', 0.20701844), ('SUV', 0.1855979), ('TURN', 0.23054521), ('VAR', 0.2833535)], 0.19412988)


  0%|          | 0/76 [00:00<?, ?it/s]

test oos ([('A2ME', 0.102465786), ('AC', 0.1764793), ('AT', 0.20257854), ('ATO', 0.197344), ('B2M', 0.04638074), ('BETA_d', 0.052178003), ('BETA_m', 0.22899298), ('C2A', 0.21090877), ('CF2B', 0.21832037), ('CF2P', 0.20767754), ('CTO', 0.20785756), ('D2A', 0.19394664), ('D2P', 0.19493258), ('DPI2A', 0.22279502), ('E2P', 0.20556602), ('FC2Y', 0.1991703), ('HIGH52', 0.08015465), ('INV', 0.044751428), ('IdioVol', 0.05725504), ('LEV', 0.20658676), ('ME', 0.17708695), ('NI', 0.20794658), ('NOA', 0.19245341), ('OA', 0.21245033), ('OL', 0.13937564), ('OP', 0.19273183), ('PCM', 0.2047311), ('PM', 0.20926191), ('PROF', 0.20979418), ('Q', 0.22187218), ('R12_2', 0.19587877), ('R12_7', 0.19468276), ('R2_1', 0.20189002), ('R36_13', 0.21197182), ('R60_13', 0.2004303), ('RNA', 0.22548553), ('ROA', 0.16615266), ('ROE', 0.22159225), ('RVAR', 0.19197169), ('S2P', 0.19198857), ('SGA2S', 0.21719897), ('SPREAD', 0.21168381), ('SUV', 0.17543414), ('TURN', 0.216192), ('VAR', 0.28491297)], 0.19237408)
